In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

/home/user2/projects/QwInSight/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load Qwen3-1.7B model and tokenizer
model_name = "Qwen/Qwen3-1.7B"
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.76it/s]


In [16]:
# Hindi prompt
prompt = "भारत की राजधानी क्या है?"
messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
print("content:", content)

content: भारत की राजधानी **नई दिल्ली** है। यह भारत के संसद और राज्यपाल के राजधानी भी है। 1947 के अपने अपने अधिकार अधिनियम के बाद नई दिल्ली के रूप में चुनी गई थी।


In [12]:
# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [13]:
# Generate output
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.95
    )

In [14]:
# Decode and print
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("💬 Model output:")
print(response)


💬 Model output:
India ka capital kya hai? 2500 words. 1. 2. 3. 4. 5. 6. 7. 8. 9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 2
